In [3]:
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error


In [6]:
import mlflow 

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc_taxi_experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>

In [7]:
df=pd.read_parquet("Data/green_tripdata_2021-01.parquet")

In [8]:
#get trip duration

df["lpep_dropoff_datetime"]=pd.to_datetime(df["lpep_dropoff_datetime"])

df["lpep_pickup_datetime"]=pd.to_datetime(df["lpep_pickup_datetime"])

df["trip_duration"]=df["lpep_dropoff_datetime"]-df["lpep_pickup_datetime"]


In [9]:
#conver to duration in minutes
df["trip_duration"]=df["trip_duration"].apply(lambda x: x.total_seconds()/60)

In [10]:
#We are going to only consider trips with duration between 1 and 60 minutes

df=df[df["trip_duration"].between(1,60)]

In [11]:
#Select what categorical and numerical variables we are going to use

categorical=["PULocationID","DOLocationID"]
numerical=["trip_distance"]


In [12]:
#one-hot encoding to categorical variables

df[categorical]=df[categorical].astype(str)

In [13]:
#Create dict from df
train_dicts=df[categorical+numerical].to_dict(orient="records")

dv=DictVectorizer()
X_train=dv.fit_transform(train_dicts)

In [14]:
#target variables
target="trip_duration"
y_train=df[target].values

In [16]:
#linear regression model
with mlflow.start_run():
    mlflow.set_tag("Developer", "FF")
    mlflow.log_param("train_data_path","January")
    mlflow.log_param("val_data_path","february")
    alpha=0.01
    mlflow.log_param("alpha",alpha)
    lr=Lasso(alpha)
    lr.fit(X_train, y_train)
    #Apply model
    y_pred=lr.predict(X_train)

    #RMSE of our baseline model
    RMSE=mean_squared_error(y_train, y_pred, squared=False)

    mlflow.log_metric("rmse", RMSE)